## Rigid body 6 DOF
Devlop a system for a rigid bofy in 6 DOF and do a simualtion

In [1]:
import warnings
#warnings.filterwarnings('ignore')
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
import sympy as sp
import sympy.physics.mechanics as me
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from substitute_dynamic_symbols import substitute_dynamic_symbols, find_name, find_derivative_name, lambdify, find_derivatives


In [35]:
x0,y0,z0 = me.dynamicsymbols('x0 y0 z0')
x01d,y01d,z01d = me.dynamicsymbols('x0 y0 z0',1)
x02d,y02d,z02d = me.dynamicsymbols('x0 y0 z0',2)

phi,theta,psi = me.dynamicsymbols('phi theta psi')
phi1d,theta1d,psi1d = me.dynamicsymbols('phi theta psi',1)
phi2d,theta2d,psi2d = me.dynamicsymbols('phi theta psi',2)


p_x = sp.Symbol('p_x')
p_y = sp.Symbol('p_y')
p_z = sp.Symbol('p_z')



In [36]:
x02d

Derivative(x0(t), (t, 2))

In [37]:
N = me.ReferenceFrame('N')

In [38]:
S = N.orientnew('S', 'Body', [psi,theta,phi],'ZYX')

In [39]:
S.ang_vel_in(N)

(-sin(theta)*psi' + phi')*S.x + (sin(phi)*cos(theta)*psi' + cos(phi)*theta')*S.y + (-sin(phi)*theta' + cos(phi)*cos(theta)*psi')*S.z

In [40]:
S.ang_acc_in(N)

(-sin(theta)*psi'' - cos(theta)*psi'*theta' + phi'')*S.x + (-sin(phi)*sin(theta)*psi'*theta' + sin(phi)*cos(theta)*psi'' - sin(phi)*phi'*theta' + cos(phi)*cos(theta)*phi'*psi' + cos(phi)*theta'')*S.y + (-sin(phi)*cos(theta)*phi'*psi' - sin(phi)*theta'' - sin(theta)*cos(phi)*psi'*theta' + cos(phi)*cos(theta)*psi'' - cos(phi)*phi'*theta')*S.z

In [41]:
M = me.Point('M')  # Origo of measurement system
M.set_vel(N, 0)

O = M.locatenew('O', x0*N.x + y0*N.y + z0*N.z) # Origo of ship
O.set_vel(N, x01d*N.x + y01d*N.y + z01d*N.z)
O.set_acc(N, x02d*N.x + y02d*N.y + z02d*N.z)



In [42]:
P = O.locatenew('P', p_x*S.x + p_y*S.y + p_z*S.z)
P.set_vel(S, 0)


In [43]:
S.dcm(N)

Matrix([
[                                      cos(psi(t))*cos(theta(t)),                                        sin(psi(t))*cos(theta(t)),            -sin(theta(t))],
[sin(phi(t))*sin(theta(t))*cos(psi(t)) - sin(psi(t))*cos(phi(t)),  sin(phi(t))*sin(psi(t))*sin(theta(t)) + cos(phi(t))*cos(psi(t)), sin(phi(t))*cos(theta(t))],
[sin(phi(t))*sin(psi(t)) + sin(theta(t))*cos(phi(t))*cos(psi(t)), -sin(phi(t))*cos(psi(t)) + sin(psi(t))*sin(theta(t))*cos(phi(t)), cos(phi(t))*cos(theta(t))]])

In [44]:
vector = P.pos_from(M)
motion_P = vector.express(N)
motion_P

(p_x*cos(psi)*cos(theta) + p_y*(sin(phi)*sin(theta)*cos(psi) - sin(psi)*cos(phi)) + p_z*(sin(phi)*sin(psi) + sin(theta)*cos(phi)*cos(psi)) + x0)*N.x + (p_x*sin(psi)*cos(theta) + p_y*(sin(phi)*sin(psi)*sin(theta) + cos(phi)*cos(psi)) + p_z*(-sin(phi)*cos(psi) + sin(psi)*sin(theta)*cos(phi)) + y0)*N.y + (-p_x*sin(theta) + p_y*sin(phi)*cos(theta) + p_z*cos(phi)*cos(theta) + z0)*N.z

In [45]:
motion_matrix = vector.to_matrix(N)
motion_matrix

Matrix([
[ p_x*cos(psi(t))*cos(theta(t)) + p_y*(sin(phi(t))*sin(theta(t))*cos(psi(t)) - sin(psi(t))*cos(phi(t))) + p_z*(sin(phi(t))*sin(psi(t)) + sin(theta(t))*cos(phi(t))*cos(psi(t))) + x0(t)],
[p_x*sin(psi(t))*cos(theta(t)) + p_y*(sin(phi(t))*sin(psi(t))*sin(theta(t)) + cos(phi(t))*cos(psi(t))) + p_z*(-sin(phi(t))*cos(psi(t)) + sin(psi(t))*sin(theta(t))*cos(phi(t))) + y0(t)],
[                                                                                            -p_x*sin(theta(t)) + p_y*sin(phi(t))*cos(theta(t)) + p_z*cos(phi(t))*cos(theta(t)) + z0(t)]])

## Mass

In [46]:
mass = sp.symbols('m')

In [47]:
c_og = me.Point('c_og')
c_og.set_vel(S,0)

## Inertia

In [48]:
I_xx, I_yy, I_zz = sp.symbols('I_xx, I_yy, I_zz')
body_inertia_dyadic = me.inertia(S, ixx=I_xx, iyy=I_yy, izz=I_zz)
body_inertia_dyadic

I_xx*(S.x|S.x) + I_yy*(S.y|S.y) + I_zz*(S.z|S.z)

In [49]:
body_inertia_dyadic.to_matrix(S)

Matrix([
[I_xx,    0,    0],
[   0, I_yy,    0],
[   0,    0, I_zz]])

In [50]:
body_central_inertia = (body_inertia_dyadic, c_og)

In [51]:
body = me.RigidBody('Rigid body', masscenter=c_og, frame = S,
                      mass=mass, inertia=body_central_inertia)

## Forces

In [52]:
fx, fy, fz, mx, my, mz = sp.symbols('f_x f_y f_z m_x m_y m_z')

In [53]:
force_vector = fx*N.x + fy*N.y + fz*N.z
torque_vector = mx*N.x + my*N.y + mz*N.z

In [56]:
force = (c_og, force_vector)
torque = (c_og, torque_vector)

## Equations of Motion

In [57]:
coordinates = [x0, y0, z0, phi, theta, psi]
speeds = [x01d, y01d, z01d, phi1d, theta1d, psi1d]

In [63]:
kinematical_differential_equations = [x01d - x0.diff(),
                                      #x02d - x01d.diff(),
                                      
                                      y01d - y0.diff(),
                                      #y02d - y01d.diff(),
                                      
                                      z01d - z0.diff(),
                                      #z02d - z01d.diff(),
                                      
                                      phi1d - phi.diff(),
                                      #phi2d - phi1d.diff(),
                                      
                                      theta1d - theta.diff(),
                                      #theta2d - theta1d.diff(),
                                      
                                      psi1d - psi.diff(),
                                      #psi2d - psi2d.diff(),                                      
                                     ]

In [64]:
kane = me.KanesMethod(N, coordinates, speeds, kinematical_differential_equations)

NotImplementedError: Underdetermined systems not supported.

In [59]:
loads = [force,
         torque]

In [60]:
bodies = [body]
fr, frstar = kane.kanes_equations(bodies, loads)

AttributeError: Create an instance of KanesMethod with kinematic differential equations to use this method.